In [73]:
import pandas as pd
from fastai import *
articleDetails = pd.read_pickle('article_details.pkl')

In [74]:
count = 0
for i in range(0, len(articleDetails['List Links'])):
    count += len(articleDetails['List Links'][i])
            
print("We have " + str(count) + " articles in our Corpus of data")

We have 3353 articles in our Corpus of data


In [75]:
headlines = []
for i in range(0, len(articleDetails['List Links'])):
    for j in range(0, len(articleDetails['List Links'][i])):
        headlines.append(articleDetails['List Links'][i][j])

headline_DF = pd.DataFrame()
headline_DF['headlines_str'] = headlines

In [103]:
# Average length of headline
sum_lens = 0
for i in range(0, len(headlines)):
    sum_lens += len(headlines[i].split())
    
average_length = int(sum_lens / len(headlines))
print(average_length)

7


In [131]:
list1 = ["Cat"]
print(' '.join(list1))
print(len(' '.join(list1)))

list2 = ["Cat", "and", "dog"]
print(' '.join(list2[:3+1]))

Cat
3
Cat and dog


In [134]:
# Most Popular first words
def find_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = True
    return return_value

def find_index_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = j
    return return_value


def generateLeadingStringArray(num_words):
    local_arr = []
    for i in range(0, len(headlines)):
        first_word_of_string = ' '.join(headlines[i].split()[:num_words])
        if find_in_list_of_tuple(local_arr, first_word_of_string) == True:
            # Word has already been added to local_arr
            local_arr[find_index_in_list_of_tuple(local_arr, first_word_of_string)][1] += 1
        else:
            local_arr.append([first_word_of_string, 1])

    local_arr = sorted(local_arr, key=lambda tup: tup[1], reverse=True)
    
    return local_arr

In [135]:
most_popular_first_word = generateLeadingStringArray(1)
most_popular_two_words = generateLeadingStringArray(2)
most_popular_three_words = generateLeadingStringArray(3)

print(most_popular_first_word[:5])
print(most_popular_two_words[:5])
print(most_popular_three_words[:5])

[['Labour', 125], ['WATCH:', 75], ['Corbyn', 60], ['BBC', 55], ['The', 42]]
[['PMQs SKETCH:', 22], ['Owen Jones', 21], ['Diane Abbott', 19], ['Read in', 15], ['Tory MP', 12]]
[['Read in Full:', 14], ['+++ RESHUFFLE LIVE', 4], ['Channel 4 News', 4], ['READ IN FULL:', 3], ['EXCLUSIVE: Home Office', 3]]


In [92]:
dls = TextDataLoaders.from_df(headline_DF, path="./", text_col='headlines_str', valid_pct=0.3, is_lm=True)

In [93]:
model = language_model_learner(dls, AWD_LSTM)

In [94]:
model.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,4.986782,4.678916,02:52
1,4.901416,4.501376,02:44
2,4.796350,4.389595,03:03
3,4.735701,4.342404,02:47
4,4.676809,4.335111,03:04


In [144]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

print()

def getGeneration(leading_string_arr, number_of_words=10, min_length=1, max_length=3, number_of_generations=3):
    model_output = ""
    model_output = model_output + "----------------\n"
    model_output = model_output + "Using the array: " + namestr(leading_string_arr, globals())[0] + "\n"
    for i in range(0, number_of_words):
        if leading_string_arr[i][0][-1] == ":":
            model_output = model_output + leading_string_arr[i][0] + "\n"
        else:
            model_output = model_output + leading_string_arr[i][0] + ":\n"
        for j in range(min_length, max_length):
            num_words = int(average_length*(j+1))
            model_output = model_output + "\t" + "With length of " + str(num_words) + ":\n"
            for k in range(0, number_of_generations):
                with model.no_bar(), model.no_logging():
                    model_output = model_output + "\t\t" + model.predict(leading_string_arr[i][0], n_words=num_words) + "\n"
            
    print(model_output)

In [143]:
getGeneration(most_popular_first_word)
getGeneration(most_popular_two_words)
getGeneration(most_popular_three_words)

---------------
Using the array: most_popular_first_word
Labour:
	With length of 14:
		Labour Stand at The End of Time Team In



---------------
Using the array: most_popular_two_words
PMQs SKETCH:
	With length of 14:
		pmqs SKETCH : Christmas Time Lies Last Time In Canada

